In [1]:
import sys
import os

from PyQt5 import QtWidgets
from PyQt5.QtWidgets import (QMainWindow, QTextEdit,
                QAction, QFileDialog, QApplication,
                QMessageBox, QTableView
                )
from PyQt5.QtGui import (QStandardItemModel,
                          QStandardItem,
                         QPalette, QColor
                          )
from PyQt5.QtCore import QRect, Qt
import cv2
import numpy as np
import xlsxwriter

import mainwindow
from model_response import ModelResponse
from model.evaluator import RoundClassifier

In [2]:
class MainWindow(QMainWindow):
    """Класс описывает поведение главного окна графического интерфейса.
    
    """
    def __init__(self):
        """Конструктор класса."""
        super(MainWindow, self).__init__()
        self.ui = mainwindow.Ui_MainWindow()
        self.ui.setupUi(self)
        self.path_predicted = ""
        self.path_expert = ""
        self.names_list_predicted = []
        self.images_list_predicted = []
        self.names_list_expert = []
        self.images_list_expert = []
        self.ui.explore_path_predicted.clicked.connect(self.get_path_predicted)
        self.ui.explore_path_expert.clicked.connect(self.get_path_expert)
        self.ui.evaluate_button.clicked.connect(self.start_evaluation)
        self.ui.save_to_xls.clicked.connect(self.save_to_xls_file)
        self.init_ui()
        
    def init_ui(self):
        """Функция устанавливает исходные настройки
        для графического интерфейса
        
        """
        self.setStyleSheet("background-color: white;") 
        self.ui.explore_path_predicted.setStyleSheet(
            'QPushButton {background-color: #92000a; color : white};'
            )
        self.ui.explore_path_expert.setStyleSheet(
            'QPushButton {background-color: #92000a; color : white};'
            )
        self.ui.evaluate_button.setStyleSheet(
            'QPushButton {background-color: #92000a; color : white};'
            )
        self.ui.save_to_xls.setStyleSheet(
            'QPushButton {background-color: #92000a; color : white};'
            )
        self.ui.resalts_table.setVisible(False)
        self.ui.label_predicted.setGeometry(20, 25, 290, 40)
        self.ui.label_expert.setGeometry(20, 80, 290, 40)
        self.ui.path_predicted_line.setGeometry(330, 25, 270, 40)
        self.ui.path_expert_line.setGeometry(330, 80, 270, 40)
        self.ui.explore_path_predicted.setGeometry(640, 25, 150, 40)
        self.ui.explore_path_expert.setGeometry(640, 80, 150, 40)
        self.ui.evaluate_button.setGeometry(20, 140, 250, 40)
        self.ui.save_to_xls.setGeometry(20, 185, 250, 40)
        
    def get_path_predicted(self):
        """Функция получает из проводника путь к папке
        с изображениями, которые нужно оценить,
        отображает путь в соответствующем
        текстовом поле и  делает текстовое 
        поле недоступным для редактирования
        
        """
        self.path_predicted = QFileDialog.getExistingDirectory(self)
        self.ui.path_predicted_line.setText(self.path_predicted)
        self.ui.path_predicted_line.setReadOnly(True)
        
    def get_path_expert(self):
        """Функция получает из проводника путь к папке
        с экспертной разметкой изображений, отображает 
        путь в соответствующем текстовом поле и 
        делает текстовое поле недоступным 
        для редактирования.
        
        """
        self.path_expert = QFileDialog.getExistingDirectory(self)
        self.ui.path_expert_line.setText(self.path_expert)
        self.ui.path_expert_line.setReadOnly(True)
    
    def get_list_of_images(self, path):
        """Функция принимает на вход путь к директории,
        обращается к директории и получает из неё файлы.
        Расширение файлов проверяется. Функция возвращает 
        2 списка: список имён картинок из данной
        директории и список самих картинок, преобразованных
        в массивы чисел.
        
        """
        valid_images = [".jpg",".gif",".png",".tga"]
        names_list = [] 
        images_list = []
        for file in os.listdir(path):
            ext = os.path.splitext(file)[1]
            if ext.lower() not in valid_images:
                continue
            names_list.append(file)
            images_list.append(cv2.imread(os.path.join(path, file), cv2.IMREAD_GRAYSCALE))
        return np.array(names_list), np.array(images_list)
    
    def start_evaluation(self):
        """Функция получает списки картинок и
        передаёт их в экземпляр класса-оболочки потока.
        Вызывает исполнение функции в отдельном потоке
        и блокирует все кнопки на интерфейсе
        
        """
        self.path_predicted = self.ui.path_predicted_line.text()
        self.path_expert = self.ui.path_expert_line.text()
        try:
            (self.names_list_predicted, 
                 self.images_list_predicted
                 ) = self.get_list_of_images(self.path_predicted)
            (self.names_list_expert, 
                 self.images_list_expert
                 ) = self.get_list_of_images(self.path_expert)
            model_response_object = ModelResponse(
                 self.images_list_predicted, self.images_list_expert
                 )
            model_response_object.throw_resalts.connect(
                 self.catch_resalts
                 )
            model_response_object.start()
            self.ui.explore_path_predicted.setEnabled(False)
            self.ui.explore_path_expert.setEnabled(False)
            self.ui.evaluate_button.setEnabled(False)
        except FileNotFoundError:
            QMessageBox.critical(self, "Ошибка ", 
                 "Некорректный путь к папке", 
                 QMessageBox.Ok
                 )
            
    def show_resalts(self, resalt):
        """Функция отображает результат
        в таблице на интерфейсе
        
        """
        model = QStandardItemModel()
        it = QStandardItem()
        horizontalHeader = []
        horizontalHeader.append("Имя картинки")
        horizontalHeader.append("Оценка")
        verticalHeader = []
        for i in range(1, len(resalt) + 1):
            verticalHeader.append(str(i))
        model.setHorizontalHeaderLabels(horizontalHeader)
        model.setVerticalHeaderLabels(verticalHeader)
        for i in range(len(resalt)):
            item = QStandardItem(self.names_list_predicted[i])
            model.setItem(i, 0, item)
            item = QStandardItem(str(resalt[i]))
            model.setItem(i, 1, item)
        self.ui.resalts_table.setModel(model)
        self.ui.resalts_table.resizeColumnsToContents()
        self.ui.resalts_table.resizeRowsToContents()
        self.ui.resalts_table.verticalHeader().setDefaultSectionSize(10)
        self.ui.resalts_table.setHorizontalScrollBarPolicy(
                Qt.ScrollBarAlwaysOff
                )
        table_geometry = self.ui.resalts_table.geometry()
        table_geometry.setWidth(self.ui.resalts_table.frameWidth() * 2 + 
                self.ui.resalts_table.verticalHeader().width() +
                self.ui.resalts_table.columnWidth(0) + 25 +
                self.ui.resalts_table.columnWidth(1)
                )
        table_geometry.setHeight(300)
        self.ui.resalts_table.setGeometry(table_geometry)
        
    
    def catch_resalts(self, resalt):
        """Функция связана с сигналом, вызывающимся
        по окончании работы модели. Получает данные
        и выводит их на интерфейс
        
        """
        self.ui.explore_path_predicted.setEnabled(True)
        self.ui.explore_path_expert.setEnabled(True)
        self.ui.evaluate_button.setEnabled(True)
        self.ui.resalts_table.setVisible(True)
        self.show_resalts(resalt)
       
    def save_to_xls_file(self):
        """Функция сохраняет результат работы модели
        в xls файл
        
        """
        fileName = QFileDialog.getSaveFileName(None, 
                        'Save Excel file', "",
                        'Excel files (*.xlsx)'
                         )
        if fileName == ('',''): 
            return
        workbook = xlsxwriter.Workbook(fileName[0])
        worksheet = workbook.add_worksheet()
        try:
            for i in range(self.ui.resalts_table.model().rowCount()):
                item = self.ui.resalts_table.model().index(i, 0).data()
                worksheet.write(i, 0, item)
                item = self.ui.resalts_table.model().index(i, 1).data()
                worksheet.write(i, 1, int(item))
            workbook.close()
        except:
            return

In [3]:
def main():
    """Функция, из которой осуществляется вызов 
    основного окна приложения
    
    """
    app = QtWidgets.QApplication([])
    app.setStyle('Fusion')
    application = MainWindow()
    application.show()
    sys.exit(app.exec())

In [ ]:
if __name__ == '__main__':
    main()